In [13]:
import numpy as np
import pandas as pd

In [26]:
reg_ratios = pd.read_csv("RegularSeasonDetailedResultsRatios.csv")
tour_ratios = pd.read_csv("NCAATourneyDetailedResultsRatios.csv")

# Get columns
all_cols = reg_ratios.columns.tolist()
cols_to_drop = ["Season", "WTeamID", "WScore", "LTeamID", "LScore", "NumTeamWon", "Loc"]
stats_columns = [item for item in all_cols if item not in cols_to_drop]

# seasons = pd.unique(reg_ratios["Season"])
seasons = [2011, 2012, 2013, 2014, 2015, 2016, 2017]
print(seasons)

[2011, 2012, 2013, 2014, 2015, 2016, 2017]


In [33]:
team_ids = pd.DataFrame(index=range(7*68), columns=["Season", "TeamID"])
for i in range(len(seasons)):
    season = seasons[i]
    team_ids_for_season = pd.unique(tour_ratios.loc[tour_ratios["Season"] == season, ["WTeamID", "LTeamID"]].values.ravel('K'))
    for j in range(len(team_ids_for_season)):
        team_ids.loc[i*68+j, "Season"] = season
        team_ids.loc[i*68+j, "TeamID"] = team_ids_for_season[j]
        
# Create a data sets to hold regular season averages for every team in the tournament for every specified year
reg_season_avg_cols = ["Season", "TeamID"] + stats_columns
reg_season_avgs = pd.DataFrame(index=range(team_ids.shape[0]), columns=reg_season_avg_cols)
reg_season_avgs["Season"] = team_ids["Season"]
reg_season_avgs["TeamID"] = team_ids["TeamID"]

for season in seasons:
    team_ids_for_season = reg_season_avgs.loc[reg_season_avgs["Season"] == season, "TeamID"].tolist()
        
    for team_id in team_ids_for_season:
        games_all = reg_ratios.loc[(reg_ratios["WTeamID"] == team_id) | (reg_ratios["LTeamID"] == team_id)]
        games = games_all.loc[games_all["Season"] == season]
        games.reset_index(inplace=True, drop=True)
        games_stats = games.loc[:, stats_columns]
        for i, row in games.iterrows():
            if ((team_id == row["WTeamID"]) & (row["NumTeamWon"] == 0)) | ((team_id == row["LTeamID"]) & (row["NumTeamWon"] == 1)):
                games_stats.iloc[i] = games_stats.iloc[i].apply(np.reciprocal)
    
        reg_season_avgs.loc[(reg_season_avgs["TeamID"] == team_id) & (reg_season_avgs["Season"] == season), stats_columns] = games_stats.mean(axis=0).ravel()

reg_season_avgs.to_csv("RegularSeasonAverageRatios.csv")